In [5]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import dialogflow
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import regex as re
import random
import requests
from bs4 import BeautifulSoup as bs

from data.keyvault import keyvault

In [6]:
###########
# Settings
###########

DATA_PATH = "data/Otvety.txt"
ANSWERS_PATH = 'data/prepared_answers.txt'
FASTTEXT_MODEL_PATH = 'data/ft_100_model.h5'
RAW_SENTENCES_PATH = 'data/raw_sentences.pkl'
INDEX_MAP_PATH = 'data/index_map.pkl'
FT_INDEX_PATH = 'data/ft_index'
FT_VECTOR_MAX_LEN = 100

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

## Рутина предобработки. Подготовка и обучение FastText модели на ответах mail.ru

In [7]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [8]:
def get_sentences(input_file, len_limit=1500000):
    
    """Process text. extract sentense tokens with morph processor"""
    
    # Preprocess for models fitting

    sentences = []

    morpher = MorphAnalyzer()
    sw = set(get_stop_words("ru"))
    exclude = set(string.punctuation)
    c = 0

    with open(input_file, "r", encoding='utf-8') as fin:
        for line in tqdm(fin):
            spls = preprocess_txt(line, exclude, morpher, sw)
            sentences.append(spls)
            c += 1
            if c > len_limit:
                break
                
    return sentences

In [6]:
sentences = get_sentences(DATA_PATH)

In [14]:
modelFT = FastText.load(FASTTEXT_MODEL_PATH)

In [20]:
# modelFT = FastText.load("ft_model.h5")

# Создание индексов FastText
ft_index = annoy.AnnoyIndex(FT_VECTOR_MAX_LEN ,'angular')

index_map = {}
counter = 0

with open(ANSWERS_PATH, "r", encoding='utf-8') as f:
    for line in tqdm(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(FT_VECTOR_MAX_LEN)
        for word in question:
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

# Сохранение просчитанной карты индексов
with open(INDEX_MAP_PATH, 'wb') as f:
    pickle.dump(index_map, f)
    
# Построеиние индексов на 20 деревьях и сохранение на диск
ft_index.build(20)
ft_index.save(FT_INDEX_PATH)

C:\Users\Nickel\anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
C:\Users\Nickel\anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


## Реализация функционала. Загрузка и тестирование бота

In [9]:
# Загрузка заготовок: Fast text модель, индексы, индексированная карта ответов

modelFT = FastText.load(FASTTEXT_MODEL_PATH)

ft_index = annoy.AnnoyIndex(FT_VECTOR_MAX_LEN, 'angular')
ft_index.load(FT_INDEX_PATH)

with open(INDEX_MAP_PATH, 'rb') as f:
    index_map = pickle.load(f)

In [10]:
def preprocess_txt(line):
    """Строковый препроцессинг,  текстовой строки строки"""
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

def get_response(question, index, model, index_map):
#     print(f'question is: {question}')
    
    """Получение ответа из модели fast text"""
    
    question = preprocess_txt(question)
    vector = np.zeros(FT_VECTOR_MAX_LEN)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5) 
    return [index_map[i] for i in answers if len(index_map[i]) < 300] # 


def web_search_answer(question:str) -> str:
    # position = 'Data scientist'
    query = question
    main_link = 'https://yandex.ru/search/'
    link = main_link + f"?text={query.replace(' ', '%20')}&lr=213"
    try:
        req = requests.get(f'{link}').text
        parsed_html = bs(req, 'html.parser')
        target_block = parsed_html.find('div',{'class':'fact-answer'})
        res = target_block.contents[0]
    except:
        res = None
    return res

In [57]:
# тестирование модуля болталки на "ответах mail.ru"
input_text = "Где мы находимся?" #   "когда выйдет халф лайф 3?"
r = get_response(input_text, ft_index, modelFT, index_map)
r

C:\Users\Nickel\anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
C:\Users\Nickel\anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


['в "кое-где", а "кое-где" "где-то там", которая находится "он туды", а это "он туды" "он там", а "он там" "у чёрта на куличиках".. \n',
 'рай?. \n',
 'Напрашивается слово в пиз...а так судя по растениям юго-запад северной америки. \n']

In [58]:
def clear_sent(text):
    
    """Очистка текста от тегов, обработка лишних знаков препинаний, смайлов и прочего мусора"""
    
    target_sent = random.choice(text)
    target_sent = re.sub(r'<[^>]*>','', target_sent)
    target_sent = re.sub(r'[\xa0]',' ', target_sent)
    target_sent = re.sub(r'.{1,2} [\n]','', target_sent)
    target_sent = re.sub(r'[\n]','', target_sent)
    target_sent = re.sub(r'\s[.]\s','', target_sent)
    target_sent = re.sub(r"\)\)\)\)", "))", target_sent)
    target_sent = re.sub(r"\)\)", ")", target_sent)
    target_sent = re.sub(r"\) \)", "))", target_sent)
    target_sent = re.sub(r',{2,}','', target_sent)
    return target_sent

In [59]:
clear_sent(r)

'в "кое-где", а "кое-где" "где-то там", которая находится "он туды", а это "он туды" "он там", а "он там" "у чёрта на куличиках"'

## Запуск бота

In [25]:
updater = Updater(token=keyvault['telegram_token'])
dispatcher = updater.dispatcher
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = keyvault['google_api_key'] # Ключ к google API dialogflow

DIALOGFLOW_PROJECT_ID = keyvault['dialogflow_project_id'] # PROJECT ID DialogFlow 
DIALOGFLOW_LANGUAGE_CODE = 'ru' # языковая группа
SESSION_ID = 'SetMyTasksBot'  # ID телеграмм бота

start_msg = 'R1D1 машет клешнёй! Бот может поддержать беседу веселой болтовнёй и отвечать на глупые вопросы'

C:\Users\Nickel\anaconda3\lib\site-packages\ipykernel_launcher.py:1: TelegramDeprecationWarning: Old Handler API is deprecated - see https://git.io/fxJuV for details
  """Entry point for launching an IPython kernel.


In [34]:
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text=start_msg)
    logging.info("Start Bot")

def textMessage(bot, update):
    
    # Начало: Поиск интентов на DialogFlow
    input_txt = update.message.text
    print(f"input_txt is: {input_txt}")
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(DIALOGFLOW_PROJECT_ID, SESSION_ID)
    text_input = dialogflow.types.TextInput(text=input_txt, language_code=DIALOGFLOW_LANGUAGE_CODE)
    query_input = dialogflow.types.QueryInput(text=text_input)
    
    try:
        response = session_client.detect_intent(session=session, query_input=query_input)
    except InvalidArgument:
         raise

    text = response.query_result.fulfillment_text
    if text and text != 'null':
        response = response.query_result.fulfillment_text
        bot.send_message(chat_id=update.message.chat_id, text=response)
    else:
        response = web_search_answer(input_txt)
        if response != None:
            bot.send_message(chat_id=update.message.chat_id, text=response)
        else:
        # Интент DialogFlow на найден --> включение режима болталки на "ответах mail.ru"
            response = get_response(input_txt, ft_index, modelFT, index_map)
            response = clear_sent(response)
            bot.send_message(chat_id=update.message.chat_id, text=response)
    print(f"Response is: {response}")

In [35]:
# Хендлеры

start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
# Добавляем хендлеры в диспатчер
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
# Начинаем поиск обновлений
updater.start_polling(clean=True)
# Останавливаем бота на Ctrl + C
updater.idle()

input_txt is: Привет
Response is: И тебе не хворать
input_txt is: Как дела на марсе?
Response is: У меня лучшая в мире работа - в чате косить под бота!
input_txt is: Ты кто?
Response is: Я великий Бендер!
input_txt is: кто создал таблицу Менделеева?
Response is: Дмитрий Иванович Менделеев
input_txt is: когда выйдет халф лайф 3?
Response is: Последняя игра во франшизе 
input_txt is: Что у тебя в голове, бот?
Response is: Да. И горжусь этим!
input_txt is: Ты бот?
Response is: Да. И горжусь этим!
input_txt is: косил косой косой косой?


C:\Users\Nickel\anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
C:\Users\Nickel\anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Response is: После кислотных дождей-испуг вороны-удобрение))
